In [1]:
print("Importing libraries ... ")
import numpy as np
import matplotlib.pyplot as plt
import os
import json
import Puissance_4_bitboard
print("Done")

Importing libraries ... 
Done


In [2]:
file_dir = os.getcwd()+"\\"
file_data = "data_set_Puissance_4.json"

In [3]:
def load_json(json_file):
    with open(json_file) as f:
        data = json.load(f)
        return data

In [4]:
data = load_json(file_dir+file_data)
(X_train, Y_train),\
(X_val, Y_val),\
(X_test, Y_test) = (data["train"]["input"],data["train"]["output"]),\
                   (data["validation"]["input"],data["validation"]["output"]),\
                   (data["test"]["input"],data["test"]["output"])

In [5]:
print("Data in training set : {}".format(len(X_train)))
print("Data in validation set : {}".format(len(X_val)))
print("Data in test set : {}".format(len(X_test)))

Data in training set : 5189933
Data in validation set : 864988
Data in test set : 864990


In [6]:
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, BatchNormalization
import tensorflow.keras.optimizers as optimizers
import optuna

In [7]:
def my_metric(model, input, output):
    return model.evaluate(input, output, verbose=0)[0]

In [8]:
print("Creating the objective ... ")
class Objective(object):
    def __init__(self, X_train, Y_train, X_val, Y_val, model_save_path):
        # Hold this implementation specific arguments as the fields of the class.
        self._X_train = X_train
        self._Y_train = Y_train
        self._X_val = X_val
        self._Y_val = Y_val
        self._model_path = model_save_path

    def __call__(self, trial):
        model_GO = Sequential([
            Conv2D(filters = 64,
                input_shape = (6,7,2),
                activation = 'relu',
                kernel_size = (5,5),
                padding = 'same',
                data_format = 'channels_last'),
            BatchNormalization(axis = 3),
            Conv2D(filters = 64,
                activation = 'relu',
                kernel_size = (5,5),
                padding = 'same',
                data_format = 'channels_last'),
            BatchNormalization(axis = 3),
            Conv2D(filters = 64,
                activation = 'relu',
                kernel_size = (5,5),
                padding = 'same',
                data_format = 'channels_last'),
            BatchNormalization(axis = 3),
            Flatten(data_format = 'channels_last'),
            Dropout(rate = 0.5),
            Dense(units = 32,
                activation='relu'),
            Dense(2, activation='softmax'),
        ])

        model_GO.compile(loss='mse',
                    optimizer='sgd',
                    metrics=['accuracy'])

        model_GO.fit(self._X_train, self._Y_train,
          batch_size=128,
          epochs=1)

        return my_metric(model_GO, self._X_val, self._Y_val)
print("Done")

Creating the objective ... 
Done


In [9]:
DATA_NB = 100000

In [11]:
new_X_train = [[[[(int(val)&(1<<(i*6+j)))==(1<<(i*6+j)) for val in X_train[k].split()] for j in range(7)] for i in range(6)] for k in range(len(X_train))]
new_X_val = [[[[(int(val)&(1<<(i*6+j)))==(1<<(i*6+j)) for val in X_val[k].split()] for j in range(7)] for i in range(6)] for k in range(len(X_val))]
new_X_test = [[[[(int(val)&(1<<(i*6+j)))==(1<<(i*6+j)) for val in X_test[k].split()] for j in range(7)] for i in range(6)] for k in range(len(X_test))]

In [12]:
new_Y_train = Y_train
new_Y_val = Y_val
new_Y_test = Y_test
print(np.array(new_X_train).shape)
print(np.array(new_Y_train).shape)

(5189933, 6, 7, 2)
(5189933, 2)


### Symetrie
Extend with the vertical axis symetrie

In [13]:
new_X_train += [[new_X_train[i][j][::-1] for j in range(len(new_X_train[i]))] for i in range(len(new_X_train))]
new_Y_train += new_Y_train

new_X_val += [[new_X_val[i][j][::-1] for j in range(len(new_X_val[i]))] for i in range(len(new_X_val))]
new_Y_val += new_Y_val

new_X_test += [[new_X_test[i][j][::-1] for j in range(len(new_X_test[i]))] for i in range(len(new_X_test))]
new_Y_test += new_Y_test

In [14]:
TIMEOUT = 30
NB_TRIALS = 0
STUDY_NAME = "TEST"
STR_PATH = "sqlite:///"+STUDY_NAME+"/"+STUDY_NAME+".db"

In [15]:
if os.path.exists(STUDY_NAME):
    print("-------------------------------------")
    print("Study {} loaded".format(STUDY_NAME))
    print("-------------------------------------")
    study = optuna.load_study(study_name=STUDY_NAME, storage=STR_PATH)

else:
    print("New study {} created".format(STUDY_NAME))
    os.mkdir(STUDY_NAME)
    study = optuna.create_study(study_name=STUDY_NAME,direction="minimize", storage=STR_PATH)

# Execute an optimization by using an `Objective` instance.
study.optimize(Objective(new_X_train,  new_Y_train, new_X_val,  new_Y_val, STUDY_NAME+"/"+STUDY_NAME), n_trials  = NB_TRIALS)

trial = study.best_trial
print("******************************")
print("******************************")
print('Accuracy: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

New study TEST created
[I 2021-04-22 21:31:34,485] A new study created in RDB with name: TEST
81093/81093 [==============================] - 2914s 36ms/step - loss: 0.2307 - accuracy: 0.5718
[I 2021-04-22 23:45:29,873] Trial 0 finished with value: 0.2179492861032486 and parameters: {}. Best is trial 0 with value: 0.2179492861032486.
******************************
******************************
Accuracy: 0.2179492861032486
Best hyperparameters: {}
